In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%matplotlib inline

In [13]:
from sklearn.model_selection import train_test_split

In [15]:
from price_predictor.trainers.train_ensamble_model import train_random_forest
from price_predictor.data_preparation.pick_or_drop_features import drop_by_name
from price_predictor.evaluate import evaluate_model

In [4]:
pd.set_option('display.max_columns', 50)

In [5]:
data = pd.read_csv('data/cars.csv')
data

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.00,False,Минская обл.,9,13,False,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.00,True,Минская обл.,12,54,False,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.00,True,Минская обл.,4,72,False,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,all,9999.00,True,Минская обл.,9,42,True,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,universal,False,owned,all,2134.11,True,Гомельская обл.,14,7,False,True,False,True,True,False,False,False,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,Chrysler,300,automatic,silver,290000,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,2750.00,True,Минская обл.,5,85,False,True,False,False,True,True,False,False,True,True,301
38527,Chrysler,PT Cruiser,mechanical,blue,321000,2004,diesel,False,diesel,2.2,hatchback,False,owned,front,4800.00,True,Брестская обл.,4,20,False,True,False,False,True,True,False,False,True,True,317
38528,Chrysler,300,automatic,blue,777957,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,4300.00,False,Минская обл.,3,63,False,True,False,False,True,True,False,False,True,True,369
38529,Chrysler,PT Cruiser,mechanical,black,20000,2001,gasoline,False,gasoline,2.0,minivan,False,owned,front,4000.00,True,Брестская обл.,7,156,False,True,False,False,False,False,False,False,False,True,490


In [6]:
data.describe()

,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,duration_listed
count,38531.000000,38531.000000,38521.000000,38531.000000,38531.000000,38531.000000,38531.000000
mean,248864.638447,2002.943734,2.055161,6639.971021,9.649062,16.306091,80.577249
std,136072.376530,8.065731,0.671178,6428.152018,6.093217,43.286933,112.826569
min,0.000000,1942.000000,0.200000,1.000000,1.000000,1.000000,0.000000
25%,158000.000000,1998.000000,1.600000,2100.000000,5.000000,2.000000,23.000000
50%,250000.000000,2003.000000,2.000000,4800.000000,8.000000,5.000000,59.000000
75%,325000.000000,2009.000000,2.300000,8990.000000,12.000000,16.000000,91.000000
max,1000000.000000,2019.000000,8.000000,50000.000000,86.000000,1861.000000,2232.000000


## Price outliers

In [7]:
cheap = data[data['price_usd'] < 2000]
cheap

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
13,Subaru,Justy,mechanical,red,49999,2001,gasoline,False,gasoline,1.3,hatchback,False,owned,all,1666.00,False,Гомельская обл.,8,94,True,False,False,False,False,False,False,False,False,False,230
26,Subaru,Outback,automatic,green,417000,1997,gasoline,False,gasoline,2.0,universal,False,owned,front,1850.00,True,Минская обл.,18,23,False,True,False,False,False,True,False,False,False,True,71
31,Subaru,Legacy,automatic,black,299000,1994,gas,True,gasoline,2.2,universal,False,owned,all,650.00,False,Гомельская обл.,9,47,False,False,False,False,False,True,False,False,True,False,85
33,Subaru,Legacy,mechanical,black,123456,1991,gasoline,False,gasoline,2.2,sedan,False,owned,all,800.00,False,Брестская обл.,6,16,False,False,False,False,False,False,False,False,True,True,20
37,Subaru,Legacy,mechanical,red,100000,1994,gasoline,False,gasoline,2.0,sedan,False,owned,all,1700.00,True,Минская обл.,6,42,False,False,False,True,False,False,False,False,False,True,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38495,Chrysler,Neon,automatic,other,229500,2002,gasoline,False,gasoline,2.0,sedan,False,owned,front,1600.00,False,Минская обл.,15,4,False,True,False,False,False,False,False,False,False,False,116
38499,Chrysler,Stratus,automatic,violet,270000,1996,gasoline,False,gasoline,2.5,sedan,False,owned,front,1185.61,True,Витебская обл.,3,2,False,True,True,False,True,True,False,False,True,True,139
38503,Chrysler,Intrepid,automatic,black,300000,1999,gasoline,False,gasoline,2.7,sedan,False,owned,front,1300.00,True,Брестская обл.,6,2,False,True,True,True,False,True,False,True,True,True,147
38520,Chrysler,Voyager,mechanical,silver,400000,1999,diesel,False,diesel,2.5,minivan,False,owned,front,1600.00,True,Минская обл.,1,38,False,True,False,False,False,False,False,False,False,False,202


In [8]:
cheap.describe()

,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,duration_listed
count,8807.000000,8807.000000,8807.000000,8807.000000,8807.000000,8807.000000,8807.000000
mean,299704.574089,1994.073805,1.790144,1118.498912,7.342909,13.819121,73.618485
std,154591.525362,5.305776,0.362027,482.638771,4.543584,35.992840,106.848649
min,1.000000,1956.000000,0.200000,1.000000,1.000000,1.000000,0.000000
25%,230000.000000,1991.000000,1.600000,700.000000,4.000000,1.000000,20.000000
50%,300000.000000,1995.000000,1.800000,1100.000000,6.000000,4.000000,54.000000
75%,357940.000000,1998.000000,2.000000,1500.000000,9.000000,13.000000,87.500000
max,1000000.000000,2014.000000,5.200000,1999.000000,54.000000,1039.000000,2049.000000


In [9]:
super_cheap = data[data['price_usd'] < 500]
super_cheap

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
240,Subaru,Legacy,automatic,grey,999999,1991,gasoline,False,gasoline,2.2,universal,False,emergency,all,400.00,False,Минская обл.,7,2,True,False,False,False,False,False,False,False,False,False,62
321,LADA,111,mechanical,other,256000,1986,gasoline,False,gasoline,1.6,sedan,False,owned,rear,420.00,True,Витебская обл.,7,9,False,False,False,False,False,False,False,False,False,True,31
326,LADA,111,mechanical,red,123456,1977,gasoline,False,gasoline,1.1,sedan,False,emergency,rear,120.00,False,Гродненская обл.,5,1,True,False,False,False,False,False,False,False,False,False,2
351,LADA,111,mechanical,red,359000,1995,gasoline,False,gasoline,1.5,hatchback,False,owned,front,474.25,False,Витебская обл.,1,1,True,False,False,False,False,False,False,False,False,False,7
358,LADA,110,mechanical,other,12345,1990,gasoline,False,gasoline,1.3,sedan,False,owned,rear,400.00,True,Гомельская обл.,5,2,True,False,False,False,False,False,False,False,False,False,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37617,Peugeot,405,mechanical,blue,137000,1993,gasoline,False,gasoline,1.6,sedan,False,emergency,front,464.76,False,Минская обл.,7,3,True,False,False,False,False,False,False,False,False,False,63
37812,Peugeot,405,mechanical,blue,460000,1989,gasoline,False,gasoline,1.6,sedan,False,owned,front,450.00,False,Минская обл.,4,13,True,False,False,False,False,False,False,False,False,False,88
37858,Peugeot,405,mechanical,red,482803,1994,gasoline,False,gasoline,1.8,universal,False,owned,front,379.40,True,Витебская обл.,11,1,True,False,False,False,False,False,False,False,False,False,74
37875,Peugeot,205,mechanical,black,240000,1992,gasoline,False,gasoline,1.2,hatchback,False,owned,front,450.00,False,Гомельская обл.,5,1,True,False,False,False,False,False,False,False,False,False,76


In [10]:
super_cheap.describe()

,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,duration_listed
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,292188.912169,1990.521693,1.659683,340.084571,6.027513,14.691005,79.214815
std,213983.537458,5.737718,0.317388,98.327146,3.820008,46.257498,156.288600
min,1.000000,1960.000000,0.800000,1.000000,1.000000,1.000000,0.000000
25%,150000.000000,1987.000000,1.400000,280.000000,4.000000,1.000000,15.000000
50%,290000.000000,1991.000000,1.600000,350.000000,5.000000,3.000000,37.000000
75%,360000.000000,1995.000000,1.800000,400.000000,8.000000,10.000000,81.000000
max,1000000.000000,2008.000000,3.000000,499.380000,39.000000,762.000000,1737.000000


In [11]:
expensive = data[data['price_usd'] > 35_000]
expensive

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
212,Subaru,Outback,automatic,other,52000,2015,gasoline,False,gasoline,2.5,suv,False,owned,all,35500.0,False,Минская обл.,6,2,False,True,False,False,True,True,False,True,True,True,57
641,Dodge,1500,automatic,red,44500,2015,gasoline,False,gasoline,3.0,pickup,False,owned,all,35900.0,False,Минская обл.,12,5,False,True,True,True,False,False,True,True,True,False,117
1218,Kia,Sorento,automatic,black,0,2019,diesel,False,diesel,2.2,suv,True,new,all,44700.0,True,Минская обл.,11,49,False,False,False,True,True,True,True,True,True,True,83
4806,Acura,MDX,automatic,black,83000,2014,gasoline,False,gasoline,3.5,suv,False,owned,all,36500.0,False,Могилевская обл.,11,5,False,True,False,True,True,True,True,True,True,True,228
4868,Lexus,RX,automatic,white,43436,2017,gasoline,False,gasoline,2.0,suv,False,owned,all,45400.0,True,Минская обл.,27,18,False,True,False,True,True,True,True,True,True,True,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35733,Mercedes-Benz,V250,automatic,violet,164000,2015,diesel,False,diesel,2.2,minivan,False,owned,all,42777.0,True,Минская обл.,13,1,False,True,True,True,True,True,True,True,True,True,50
35769,Mercedes-Benz,SL350,automatic,grey,62000,2012,gasoline,False,gasoline,3.0,cabriolet,False,owned,rear,39900.0,True,Минская обл.,14,93,False,True,True,True,True,True,True,True,True,True,254
36081,Mercedes-Benz,GL400,automatic,grey,87000,2014,gasoline,False,gasoline,3.0,suv,False,owned,all,42000.0,True,Минская обл.,12,2,False,True,True,True,True,True,True,True,True,True,141
36180,Mercedes-Benz,E63 AMG,automatic,white,60000,2013,gasoline,False,gasoline,5.5,sedan,False,owned,all,39900.0,True,Брестская обл.,14,83,False,True,True,True,True,True,True,True,True,True,252


In [21]:
reduced_data = data[(data['price_usd'] > 500) & (data['price_usd'] < 35_000)]
reduced_data

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.00,False,Минская обл.,9,13,False,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.00,True,Минская обл.,12,54,False,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.00,True,Минская обл.,4,72,False,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,all,9999.00,True,Минская обл.,9,42,True,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,universal,False,owned,all,2134.11,True,Гомельская обл.,14,7,False,True,False,True,True,False,False,False,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,Chrysler,300,automatic,silver,290000,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,2750.00,True,Минская обл.,5,85,False,True,False,False,True,True,False,False,True,True,301
38527,Chrysler,PT Cruiser,mechanical,blue,321000,2004,diesel,False,diesel,2.2,hatchback,False,owned,front,4800.00,True,Брестская обл.,4,20,False,True,False,False,True,True,False,False,True,True,317
38528,Chrysler,300,automatic,blue,777957,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,4300.00,False,Минская обл.,3,63,False,True,False,False,True,True,False,False,True,True,369
38529,Chrysler,PT Cruiser,mechanical,black,20000,2001,gasoline,False,gasoline,2.0,minivan,False,owned,front,4000.00,True,Брестская обл.,7,156,False,True,False,False,False,False,False,False,False,True,490


In [27]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 1733.7002791446748,
 'mae': 1032.7462187783183,
 'r2': 0.9065693630361801}